In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats

from IPython.display import display
from IPython import get_ipython
get_ipython().run_line_magic('matplotlib', 'inline')

In [2]:
from sklearn import linear_model
from sklearn.tree import DecisionTreeClassifier

from tqdm import tqdm 
from time import time

In [3]:
%run ../src/load_data.py

#### One Hot Encoding

In [4]:
housing_cat_df = housing_df.select_dtypes(include = ['category'])

In [5]:
dum_cat_df = pd.get_dummies(housing_cat_df,prefix_sep='_')

In [6]:
final_feature_df = pd.merge(dum_cat_df, numeric_log_sc_df, left_index=True, right_index=True)

In [7]:
final_feature_df.head()

,MSSubClass_20,MSSubClass_30,MSSubClass_40,MSSubClass_45,MSSubClass_50,MSSubClass_60,MSSubClass_70,MSSubClass_75,MSSubClass_80,MSSubClass_85,...,GarageYrBlt,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,ThreeSsnPorch,ScreenPorch,PoolArea,MiscVal,YrSold
Id,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,1,0,0,0,0,...,1.027172,0.352403,-0.752672,0.220671,-0.359239,-0.116664,-0.271013,-0.068882,-0.087931,0.138937
2,1,0,0,0,0,0,0,0,0,0,...,-0.098465,-0.058482,1.623025,-0.702730,-0.359239,-0.116664,-0.271013,-0.068882,-0.087931,-0.613295
3,0,0,0,0,0,1,0,0,0,0,...,0.943792,0.632552,-0.752672,-0.066946,-0.359239,-0.116664,-0.271013,-0.068882,-0.087931,0.138937
4,0,0,0,0,0,0,1,0,0,0,...,0.818721,0.791303,-0.752672,-0.172910,4.091881,-0.116664,-0.271013,-0.068882,-0.087931,-1.365527
5,0,0,0,0,0,1,0,0,0,0,...,0.902101,1.697119,0.777979,0.568838,-0.359239,-0.116664,-0.271013,-0.068882,-0.087931,0.138937


In [8]:
final_feature_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1451 entries, 1 to 1460
Columns: 382 entries, MSSubClass_20 to YrSold
dtypes: float64(23), uint8(359)
memory usage: 780.8 KB


#### Decision Tree

In [9]:
from sklearn.tree import DecisionTreeClassifier

In [10]:
dtree = DecisionTreeClassifier()

In [11]:
y = housing_df['SalePrice']

In [12]:
X = final_feature_df

In [13]:
dtree.fit(X,y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [14]:
important_feature = dtree.feature_importances_
features = final_feature_df.columns
important_feature = pd.Series(important_feature.T.ravel(), index=features)

In [15]:
sort_imp_feat = important_feature.sort_values(ascending = False)
print ('\n Top 10 Features')
sort_imp_feat.head(10)


 Top 10 Features


GarageArea      0.039017
BsmtUnfSF       0.031152
LotArea         0.030987
GrLivArea       0.026226
FirstFlrSF      0.025258
YearBuilt       0.025081
TotalBsmtSF     0.024287
YearRemodAdd    0.022691
LotFrontage     0.019982
YrSold          0.019519
dtype: float64

#### Random Forest

In [16]:
from sklearn.ensemble import RandomForestClassifier

In [17]:
rfc = RandomForestClassifier(n_estimators=500)

In [18]:
rfc.fit(X,y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [19]:
rfc_important_feature = rfc.feature_importances_
rfc_important_feature = pd.Series(rfc_important_feature.T.ravel(), index=features)

In [20]:
sortrfc_imp_feat = rfc_important_feature.sort_values(ascending = False)
print ('\n Top 10 Features')
sortrfc_imp_feat.head(10)


 Top 10 Features


LotArea        0.024778
GrLivArea      0.024322
FirstFlrSF     0.023609
TotalBsmtSF    0.023186
BsmtUnfSF      0.022877
GarageArea     0.022779
LotFrontage    0.021148
YearBuilt      0.020517
GarageYrBlt    0.020139
BsmtFinSF1     0.019491
dtype: float64